<a href="https://colab.research.google.com/github/AireesRondain/PM/blob/main/PM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlxtend --upgrade
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks/paymaya

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Colab Notebooks/paymaya


In [ ]:
df = pd.read_csv('Data Analyst_Assessment_v2.csv')

In [ ]:
df.head()

,customer_number,use_case,registration_month,kyc_status,spend_volume,spend_transaction_count
0,10000003,Pay with PayMaya,12/1/20 0:00,1,3865.63,2
1,10000003,Bills Payment,12/1/20 0:00,1,836.00,1
2,10000003,PAYMAYA SHOP,12/1/20 0:00,1,3562.00,13
3,10000010,QR,8/1/18 0:00,1,214.00,1
4,10000010,Online Purchase,8/1/18 0:00,1,549.00,1


In [ ]:
df.tail()

,customer_number,use_case,registration_month,kyc_status,spend_volume,spend_transaction_count
2495,10024476,P2P Sending,11/1/19 0:00,1,900.0,1
2496,1002448,PAYMAYA SHOP,6/1/20 0:00,1,100.0,1
2497,10024501,Pay with PayMaya,12/1/19 0:00,1,1399.0,1
2498,10024501,PAYMAYA SHOP,12/1/19 0:00,1,1206.0,13
2499,10024516,PAYMAYA SHOP,10/1/18 0:00,1,200.0,1


In [ ]:
#convert customer_number to string
df['customer_number'] = df['customer_number'].astype(str) 

In [ ]:
#dataframe to build the recommendation system
txns = df[['customer_number', 'use_case']]
txns.head()

,customer_number,use_case
0,10000003,Pay with PayMaya
1,10000003,Bills Payment
2,10000003,PAYMAYA SHOP
3,10000010,QR
4,10000010,Online Purchase


Show each customer with a list of use_cases availed

In [ ]:
def string_list(x):
  return [str(i) for i in x]

txns = txns.groupby('customer_number')['use_case'].apply(list).reset_index()
txns.head()

,customer_number,use_case
0,10000003,"[Pay with PayMaya, Bills Payment, PAYMAYA SHOP]"
1,10000010,"[QR, Online Purchase, PAYMAYA SHOP, P2P Sendin..."
2,10000039,[Pay with PayMaya]
3,10000057,[Instapay Sending]
4,10000067,[PAYMAYA SHOP]


In [ ]:
uc_group = df.groupby('customer_number')['use_case'].apply(list).reset_index()
uc_group["num_use_case"] = kyc_group['use_case'].apply(lambda x: len(x))
uc_group.head()

,customer_number,use_case,num_use_case
0,10000003,"[Pay with PayMaya, Bills Payment, PAYMAYA SHOP]",3
1,10000010,"[QR, Online Purchase, PAYMAYA SHOP, P2P Sendin...",5
2,10000039,[Pay with PayMaya],1
3,10000057,[Instapay Sending],1
4,10000067,[PAYMAYA SHOP],1


In [ ]:
uc_group.tail()

,customer_number,use_case,num_use_case
1254,10024451,"[Instapay Sending, Bills Payment]",2
1255,10024476,"[Online Purchase, Instapay Sending, PAYMAYA SH...",4
1256,1002448,[PAYMAYA SHOP],1
1257,10024501,"[Pay with PayMaya, PAYMAYA SHOP]",2
1258,10024516,[PAYMAYA SHOP],1


In [ ]:
uc_group.sort_values('num_use_case',ascending=False)

,customer_number,use_case,num_use_case
68,10001240,"[P2P Sending, Pay with PayMaya, P2P - Consumer...",8
448,10008786,"[Instapay Sending, PAYMAYA SHOP, QR, Online Pu...",7
1169,10022749,"[Online Purchase, Bills Payment, PAYMAYA SHOP,...",7
26,10000689,"[PAYMAYA SHOP, QR, POS Purchase, ATM Withdrawa...",7
344,10006577,"[Pay with PayMaya, QR, PAYMAYA SHOP, Online Pu...",7
...,...,...,...
596,10011432,[PAYMAYA SHOP],1
598,10011457,[Pay with PayMaya],1
600,10011463,[Bills Payment],1
601,10011473,[P2P Sending],1


In [ ]:
#convert customer_number to string
uc_group['customer_number'] = uc_group['customer_number'].astype(str) 

In [ ]:
left_join = pd.merge(df,uc_group, on = 'customer_number', how='left')

In [ ]:
left_join

,customer_number,use_case_x,registration_month,kyc_status,spend_volume,spend_transaction_count,use_case_y,num_use_case
0,10000003,Pay with PayMaya,12/1/20 0:00,1,3865.63,2,"[Pay with PayMaya, Bills Payment, PAYMAYA SHOP]",3
1,10000003,Bills Payment,12/1/20 0:00,1,836.00,1,"[Pay with PayMaya, Bills Payment, PAYMAYA SHOP]",3
2,10000003,PAYMAYA SHOP,12/1/20 0:00,1,3562.00,13,"[Pay with PayMaya, Bills Payment, PAYMAYA SHOP]",3
3,10000010,QR,8/1/18 0:00,1,214.00,1,"[QR, Online Purchase, PAYMAYA SHOP, P2P Sendin...",5
4,10000010,Online Purchase,8/1/18 0:00,1,549.00,1,"[QR, Online Purchase, PAYMAYA SHOP, P2P Sendin...",5
...,...,...,...,...,...,...,...,...
2495,10024476,P2P Sending,11/1/19 0:00,1,900.00,1,"[Online Purchase, Instapay Sending, PAYMAYA SH...",4
2496,1002448,PAYMAYA SHOP,6/1/20 0:00,1,100.00,1,[PAYMAYA SHOP],1
2497,10024501,Pay with PayMaya,12/1/19 0:00,1,1399.00,1,"[Pay with PayMaya, PAYMAYA SHOP]",2
2498,10024501,PAYMAYA SHOP,12/1/19 0:00,1,1206.00,13,"[Pay with PayMaya, PAYMAYA SHOP]",2


In [ ]:
#save to csv
left_join.to_csv('paymaya_dataset.csv', index=False)

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

In [ ]:
te = TransactionEncoder()

te.fit(txns['use_case'])
txns_1hot = te.transform(txns['use_case'])

In [ ]:
txns_1hot = pd.DataFrame(txns_1hot, columns = te.columns_)
txns_1hot.head()

,ATM Withdrawal,Bills Payment,Instapay Sending,Online Purchase,P2P - Consumer to Negosyo Sending,P2P Sending,PAYMAYA SHOP,POS Purchase,Pay with PayMaya,QR
0,False,True,False,False,False,False,True,False,True,False
1,False,False,True,True,False,True,True,False,False,True
2,False,False,False,False,False,False,False,False,True,False
3,False,False,True,False,False,False,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False


In [ ]:
txns_1hot.tail()

,ATM Withdrawal,Bills Payment,Instapay Sending,Online Purchase,P2P - Consumer to Negosyo Sending,P2P Sending,PAYMAYA SHOP,POS Purchase,Pay with PayMaya,QR
1254,False,True,True,False,False,False,False,False,False,False
1255,False,False,True,True,False,True,True,False,False,False
1256,False,False,False,False,False,False,True,False,False,False
1257,False,False,False,False,False,False,True,False,True,False
1258,False,False,False,False,False,False,True,False,False,False


10 columns x 1258 rows (total)

Using FP Growth (Frequency-Pattern Growth) Algorithm

In [ ]:
!pip install fp-growth

In [ ]:
%%timeit -n1 -r1

fpgrowth(txns_1hot, min_support=0.05, max_len=3, use_colnames=True)

1 loop, best of 1: 14.9 ms per loop


In [ ]:
is_fp = fpgrowth(txns_1hot, min_support=0.05, max_len=5, use_colnames=True)
is_fp.sort_values('support',ascending=False)

,support,itemsets
0,0.537728,(PAYMAYA SHOP)
3,0.344718,(Instapay Sending)
1,0.274027,(Bills Payment)
4,0.209690,(Online Purchase)
5,0.204925,(P2P Sending)
2,0.190627,(Pay with PayMaya)
15,0.186656,"(PAYMAYA SHOP, Instapay Sending)"
9,0.152502,"(Bills Payment, PAYMAYA SHOP)"
10,0.110405,"(Bills Payment, Instapay Sending)"
19,0.104051,"(PAYMAYA SHOP, P2P Sending)"


##Calculate Association Rules

In [ ]:
from mlxtend.frequent_patterns import association_rules

In [ ]:
rules = association_rules(is_fp, metric="lift", min_threshold=1.1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bills Payment),(Instapay Sending),0.274027,0.344718,0.110405,0.402899,1.168777,0.015943,1.097438
1,(Instapay Sending),(Bills Payment),0.344718,0.274027,0.110405,0.320276,1.168777,0.015943,1.068042
2,"(Bills Payment, PAYMAYA SHOP)",(Instapay Sending),0.152502,0.344718,0.073074,0.479167,1.390025,0.020504,1.258141
3,"(Bills Payment, Instapay Sending)",(PAYMAYA SHOP),0.110405,0.537728,0.073074,0.661871,1.230864,0.013706,1.367144
4,"(PAYMAYA SHOP, Instapay Sending)",(Bills Payment),0.186656,0.274027,0.073074,0.391489,1.428652,0.021925,1.193033
5,(Bills Payment),"(PAYMAYA SHOP, Instapay Sending)",0.274027,0.186656,0.073074,0.266667,1.428652,0.021925,1.109105
6,(PAYMAYA SHOP),"(Bills Payment, Instapay Sending)",0.537728,0.110405,0.073074,0.135894,1.230864,0.013706,1.029497
7,(Instapay Sending),"(Bills Payment, PAYMAYA SHOP)",0.344718,0.152502,0.073074,0.211982,1.390025,0.020504,1.075480
8,(Online Purchase),(Instapay Sending),0.209690,0.344718,0.081811,0.390152,1.131799,0.009527,1.074500
9,(Instapay Sending),(Online Purchase),0.344718,0.209690,0.081811,0.237327,1.131799,0.009527,1.036237


In [ ]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(Bills Payment),(Instapay Sending),0.274027,0.344718,0.110405,0.402899,1.168777,0.015943,1.097438,1
1,(Instapay Sending),(Bills Payment),0.344718,0.274027,0.110405,0.320276,1.168777,0.015943,1.068042,1
2,"(Bills Payment, PAYMAYA SHOP)",(Instapay Sending),0.152502,0.344718,0.073074,0.479167,1.390025,0.020504,1.258141,2
3,"(Bills Payment, Instapay Sending)",(PAYMAYA SHOP),0.110405,0.537728,0.073074,0.661871,1.230864,0.013706,1.367144,2
4,"(PAYMAYA SHOP, Instapay Sending)",(Bills Payment),0.186656,0.274027,0.073074,0.391489,1.428652,0.021925,1.193033,2
5,(Bills Payment),"(PAYMAYA SHOP, Instapay Sending)",0.274027,0.186656,0.073074,0.266667,1.428652,0.021925,1.109105,1
6,(PAYMAYA SHOP),"(Bills Payment, Instapay Sending)",0.537728,0.110405,0.073074,0.135894,1.230864,0.013706,1.029497,1
7,(Instapay Sending),"(Bills Payment, PAYMAYA SHOP)",0.344718,0.152502,0.073074,0.211982,1.390025,0.020504,1.075480,1
8,(Online Purchase),(Instapay Sending),0.209690,0.344718,0.081811,0.390152,1.131799,0.009527,1.074500,1
9,(Instapay Sending),(Online Purchase),0.344718,0.209690,0.081811,0.237327,1.131799,0.009527,1.036237,1


In [ ]:
#save to csv
df.to_csv('paymaya_lift.csv', index=False)

In [ ]:
# If I'm interested in the rules that satisfy the ff: criteria:
rules[ (rules['antecedent_len'] >= 1) &
       (rules['confidence'] > 0.60) &
       (rules['lift'] > 1.1) ]

# 66% of the time if (bills payment + Instapay sending) -> Paymaya Shop

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
3,"(Bills Payment, Instapay Sending)",(PAYMAYA SHOP),0.110405,0.537728,0.073074,0.661871,1.230864,0.013706,1.367144,2
14,"(Instapay Sending, P2P Sending)",(PAYMAYA SHOP),0.086577,0.537728,0.058777,0.678899,1.262532,0.012222,1.439646,2


In [ ]:
rules[rules['antecedents'] == {'PAYMAYA SHOP'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
6,(PAYMAYA SHOP),"(Bills Payment, Instapay Sending)",0.537728,0.110405,0.073074,0.135894,1.230864,0.013706,1.029497,1
15,(PAYMAYA SHOP),"(Instapay Sending, P2P Sending)",0.537728,0.086577,0.058777,0.109306,1.262532,0.012222,1.025518,1
